## Imports, clients

In [ ]:
pip install spotipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 236 kB 8.2 MB/s 
     |████████████████████████████████| 140 kB 60.3 MB/s 
     |████████████████████████████████| 62 kB 1.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [ ]:
import pandas as pd
import requests as req
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth

In [ ]:
# ids
cid = "your cid here"
secret = "your secret here"

In [ ]:
auth_manager = SpotifyClientCredentials(client_id = cid, client_secret = secret)
sp = spotipy.Spotify(auth_manager = auth_manager)

In [ ]:
playlists = {
    "18" : "37i9dQZF1EjhpGcGKQnobp",
    "19" : "37i9dQZF1Etg0l0KogChpn",
    "20" : "37i9dQZF1ELVmD3XUmQnZ4",
    "21" : "1UhGema8MHN5T177jFJQTQ",
    "22" : "2e7e5z7Uw8VNHON860aUGq" 
}

In [ ]:
playlist_info = []

for i in playlists.keys():
  playlist_info += sp.playlist_items(playlists[i], fields = "items")["items"]

len_play = len(playlist_info)

In [ ]:
tracks = pd.DataFrame()
tracks["track_id"] = [x["track"]["id"] for x in playlist_info]
tracks["playlist_id"] = [list(playlists.keys())[int(x / len_play * 5)] for x in range(len_play)]
tracks["track_name"] = [x["track"]["name"] for x in playlist_info]

In [ ]:
artists = pd.DataFrame()
artists["artist_id"] = [x["track"]["artists"][0]["id"] for x in playlist_info]
artists["artist_name"] = [x["track"]["artists"][0]["name"] for x in playlist_info]

In [ ]:
audio_features = []
artists_info = []

for x in range(int(len(playlist_info) / 50)):
  start = x * 50
  end = start + 50
  c_tracks = tracks["track_id"][start : end]
  c_artists = artists["artist_id"][start : end]

  audio_features += sp.audio_features(c_tracks)
  artists_info += sp.artists(c_artists)["artists"]

In [ ]:
artists["genre"] =  list(map(lambda x: x["genres"][-1] if len(x["genres"]) != 0 else "N/A", artists_info))
artists["artist_img"] = list(map(lambda x: x["images"][1]["url"] if len(x["images"]) != 0 else "N/A", artists_info))
artists = artists.drop_duplicates()

In [ ]:
tracks["danceability"] = [x["danceability"] for x in audio_features]
tracks["energy"] = [x["energy"] for x in audio_features]
tracks["valence"] = [x["valence"] for x in audio_features]

In [ ]:
albums = pd.DataFrame()
albums["album_id"] = [x["track"]["album"]["id"] for x in playlist_info]
albums["album_name"] = [x["track"]["album"]["name"] for x in playlist_info]
albums["album_img"] = [x["track"]["album"]["images"][1]["url"] for x in playlist_info]
albums = albums.drop_duplicates()

In [ ]:
# save dfs to drive as csvs